## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-20-03-47-54 +0000,bbc,Trump signs bill ordering justice department t...,https://www.bbc.com/news/articles/cly2y40lj5go...
1,2025-11-20-03-35-00 +0000,wsj,Congress’s approval of legislation to force a ...,https://www.wsj.com/politics/policy/epstein-fi...
2,2025-11-20-03-30-00 +0000,wsj,The House Ethics Committee said Wednesday it w...,https://www.wsj.com/politics/ethics-probe-of-r...
3,2025-11-20-03-22-16 +0000,nyt,"For Victims of Sexual Assault on Cruise Ships,...",https://www.nytimes.com/2025/11/19/travel/crui...
4,2025-11-20-03-21-00 +0000,wsj,Trump is restarting attacks on states trying t...,https://www.wsj.com/politics/policy/trump-weig...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2337/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
0,trump,40
7,epstein,22
23,house,21
21,new,20
192,saudi,15


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
31,2025-11-20-01-59-00 +0000,wsj,President Trump signed legislation authorizing...,https://www.wsj.com/politics/policy/trump-sign...,115
14,2025-11-20-02-34-06 +0000,nypost,Trump reveals when he will meet NYC Mayor-elec...,https://nypost.com/2025/11/19/us-news/trump-re...,112
38,2025-11-20-01-39-01 +0000,startribune,Trump signs bill to release Jeffrey Epstein ca...,https://www.startribune.com/trump-signs-bill-t...,109
0,2025-11-20-03-47-54 +0000,bbc,Trump signs bill ordering justice department t...,https://www.bbc.com/news/articles/cly2y40lj5go...,108
47,2025-11-20-01-00-00 +0000,wsj,By feting Saudi Crown Prince Mohammed bin Salm...,https://www.wsj.com/politics/policy/trump-puts...,104


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
31,115,2025-11-20-01-59-00 +0000,wsj,President Trump signed legislation authorizing...,https://www.wsj.com/politics/policy/trump-sign...
120,78,2025-11-19-21-43-13 +0000,nypost,"Elon Musk, Nvidia unveil deal with Saudi state...",https://nypost.com/2025/11/19/business/elon-mu...
14,72,2025-11-20-02-34-06 +0000,nypost,Trump reveals when he will meet NYC Mayor-elec...,https://nypost.com/2025/11/19/us-news/trump-re...
79,54,2025-11-19-23-21-58 +0000,nypost,New England serial killer with haunting past r...,https://nypost.com/2025/11/19/us-news/new-engl...
222,44,2025-11-19-16-32-56 +0000,nypost,"Trump Media stock crashes to all-time lows, wi...",https://nypost.com/2025/11/19/business/trump-m...
123,42,2025-11-19-21-39-38 +0000,bbc,Children among 26 killed in one of Russia's de...,https://www.bbc.com/news/articles/cy0yere4k0zo...
2,41,2025-11-20-03-30-00 +0000,wsj,The House Ethics Committee said Wednesday it w...,https://www.wsj.com/politics/ethics-probe-of-r...
47,40,2025-11-20-01-00-00 +0000,wsj,By feting Saudi Crown Prince Mohammed bin Salm...,https://www.wsj.com/politics/policy/trump-puts...
267,33,2025-11-19-14-04-00 +0000,wsj,EU Proposes Multibillion-Euro Plan to Boost Mi...,https://www.wsj.com/world/europe/eu-proposes-m...
10,33,2025-11-20-02-51-39 +0000,wapo,Larry Summers steps back from teaching as Harv...,https://www.washingtonpost.com/nation/2025/11/...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
